In [29]:
import os
import caption
import torch
import torch.nn.functional as F
import numpy as np
import json
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import skimage.transform
import argparse
from scipy.misc import imread, imresize
from PIL import Image
from encoder import Encoder
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import cv2

In [30]:
myenc = Encoder()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0320 23:19:03.883181 140330106455872 tf_logging.py:115] Saver not created because there are no variables in the graph to restore


cuda


In [36]:
def generate_flipped_images(directory):
    for j in range(1,7):
        filename = directory + "/" + str(j) + ".jpg"
        img = cv2.imread(filename)
        both_img = img.copy()
        both_img = cv2.flip( img, -1 )
        cv2.imwrite(directory + "/" + str(j) + "_flipped.jpg" ,both_img)
    return 0

def generate_blurred_images(directory):
    for j in range(1, 7):
        filename = directory + "/" + str(j) + ".jpg"
        img = cv2.imread(filename)
        blurred_image = img.copy()
        blurred_image = cv2.blur(img, (20, 20))
        cv2.imwrite(directory + "/" + str(j) + "_blurred.jpg" ,blurred_image)
    return 0

In [34]:
def clear(raw, stem=False):
    
    f = open("./stop.txt", 'r')
    stop = f.read().split('\n')
    stopWords = set(stopwords.words('english'))
    stopWords |= set(stop)
    
    def stem_filter(sentence):
            return " ".join(wordnet_lemmatizer.lemmatize(word.lower(), pos='v') for word in word_tokenize(sentence) if word.lower() not in stopWords)

    def tagged_filter(sentence):
            tagged_sent = nltk.pos_tag(word_tokenize(sentence))
            selected = ['CD', 'FW', 'JJ', 'NN', 'NNP', 'NNS', 'NNPS', 'VBG']
            return " ".join([word[0] for word in tagged_sent if word[1] in selected])
    
    return stem_filter(raw) if stem else tagged_filter(raw)

def directory_to_sentence_matrix_clear(directory):
    sentence_dict = {}
    for j in range(1,7):
        filename = directory + "/" + str(j) + ".jpg"
        checkpoint = torch.load('/home/yerlan/HackNU/Jarvis/BEST_checkpoint_coco_5_cap_per_img_5_min_word_freq.pth.tar')
        decoder = checkpoint['decoder']
        decoder = decoder.to(device)
        decoder.eval()
        encoder = checkpoint['encoder']
        encoder = encoder.to(device)
        encoder.eval()

        # Load word map (word2ix)
        with open('/home/yerlan/HackNU/Jarvis/WORDMAP_coco_5_cap_per_img_5_min_word_freq.json' , 'r') as t:
            word_map = json.load(t)
        rev_word_map = {v: k for k, v in word_map.items()}  # ix2word


        sentence_array = []
        # Encode, decode with attention and beam search
        for i in range(1, 6):
            seq, alphas = caption.caption_image_beam_search(encoder, decoder, filename, word_map, i)
            alphas = torch.FloatTensor(alphas)

            # Visualize caption and attention of best sequence
            sentence_array.append(clear(caption.return_sentence(filename, seq, alphas, rev_word_map)))
        sentence_dict[j] = sentence_array

    return sentence_dict

def directory_to_sentence_matrix_clear_flipped(directory):
    sentence_dict = {}
    for j in range(1,7):
        filename = directory + "/" + str(j) + "_flipped.jpg"
        checkpoint = torch.load('/home/yerlan/HackNU/Jarvis/BEST_checkpoint_coco_5_cap_per_img_5_min_word_freq.pth.tar')
        decoder = checkpoint['decoder']
        decoder = decoder.to(device)
        decoder.eval()
        encoder = checkpoint['encoder']
        encoder = encoder.to(device)
        encoder.eval()

        # Load word map (word2ix)
        with open('/home/yerlan/HackNU/Jarvis/WORDMAP_coco_5_cap_per_img_5_min_word_freq.json' , 'r') as t:
            word_map = json.load(t)
        rev_word_map = {v: k for k, v in word_map.items()}  # ix2word


        sentence_array = []
        # Encode, decode with attention and beam search
        for i in range(1, 6):
            seq, alphas = caption.caption_image_beam_search(encoder, decoder, filename, word_map, i)
            alphas = torch.FloatTensor(alphas)

            # Visualize caption and attention of best sequence
            sentence_array.append(clear(caption.return_sentence(filename, seq, alphas, rev_word_map)))
        sentence_dict[j] = sentence_array

    return sentence_dict

def directory_to_sentence_matrix_clear_blurred(directory):
    sentence_dict = {}
    for j in range(1,7):
        filename = directory + "/" + str(j) + "_blurred.jpg"
        checkpoint = torch.load('/home/yerlan/HackNU/Jarvis/BEST_checkpoint_coco_5_cap_per_img_5_min_word_freq.pth.tar')
        decoder = checkpoint['decoder']
        decoder = decoder.to(device)
        decoder.eval()
        encoder = checkpoint['encoder']
        encoder = encoder.to(device)
        encoder.eval()

        # Load word map (word2ix)
        with open('/home/yerlan/HackNU/Jarvis/WORDMAP_coco_5_cap_per_img_5_min_word_freq.json' , 'r') as t:
            word_map = json.load(t)
        rev_word_map = {v: k for k, v in word_map.items()}  # ix2word


        sentence_array = []
        # Encode, decode with attention and beam search
        for i in range(1, 6):
            seq, alphas = caption.caption_image_beam_search(encoder, decoder, filename, word_map, i)
            alphas = torch.FloatTensor(alphas)

            # Visualize caption and attention of best sequence
            sentence_array.append(clear(caption.return_sentence(filename, seq, alphas, rev_word_map)))
        sentence_dict[j] = sentence_array

    return sentence_dict

In [35]:
def cos_matrix(generated_matrix_of_sentences, given_sentence):
    #For generating matrix of cosine similarities
    #myenc = Encoder()
    emgiven = myenc.encode(given_sentence)
    res = []
    for i in range(len(generated_matrix_of_sentences)):
        emgenerated = myenc.encode(generated_matrix_of_sentences[i])
        a = myenc.cosine_similarity(emgiven, emgenerated)[0][0]
        res.append(a)
        
        #For getting the average of the matrix
        res_np = np.array(res)
        mean = np.mean(res_np)
        minimum = np.min(res_np)
        
    return mean, minimum


def read_file(directory):
    with open(directory) as file:
        text = file.read()
        sentence = text.translate({ord(i):None for i in '.!@#$?]}{[,\n'}) #remove all unnecessary elements
    return sentence

In [41]:
%%time
count = 0
N = 26
for i in range(1, N):
    path = "/home/yerlan/HackNU/Jarvis/images/"
    sentence_matrix = []
    directory = path + str(i)
    # remove after first run
#     generate_flipped_images(directory)
#     generate_blurred_images(directory)
    sentence_dictionary_clear = directory_to_sentence_matrix_clear(directory)
#     sentence_dictionary_clear_flipped = directory_to_sentence_matrix_clear_flipped(directory)
#     sentence_dictionary_clear_blurred = directory_to_sentence_matrix_clear_blurred(directory)

    sent_dir = directory + "/input"
    orig_sent = read_file(sent_dir)
    res = []
    for k in range(len(sentence_dictionary_clear)):
        cos = cos_matrix(sentence_dictionary_clear[k+1], orig_sent)
        res.append((str(k + 1), cos[0]))
#     for k in range(len(sentence_dictionary_clear_flipped)):
#         cos_flipped = cos_matrix(sentence_dictionary_clear_flipped[k + 1], orig_sent)
#         res.append((str(k + 7), cos_flipped[0]))
#     for k in range(len(sentence_dictionary_clear_blurred)):
#         cos_blurred = cos_matrix(sentence_dictionary_clear_blurred[k + 1], orig_sent)
#         res.append((str(k + 13), cos_blurred[0]))

    label = sorted(res, key=lambda p: p[1])[0][0]
    while(int(label) > 6):
        label = str(int(label) - 6)
    output = directory + "/output"
    f = open(output, 'r')
    true_label = f.read().strip()
    print(label, true_label)
    if(label == true_label):
        count += 1
        
accuracy = (float(count) / (N - 1)) * 100
print(accuracy)

/home/yerlan/anaconda3/envs/deeplearning/lib/python3.6/site-packages/torch/serialization.py:435: SourceChangeWarning: source code of class 'models.Encoder' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/yerlan/anaconda3/envs/deeplearning/lib/python3.6/site-packages/torch/serialization.py:435: SourceChangeWarning: source code of class 'torch.nn.modules.container.Sequential' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/yerlan/anaconda3/envs/deeplearning/lib/python3.6/site-packages/torch/serialization.py:435: SourceChangeWarning: source code of class 'torch.nn.modules.conv.Conv2d' has changed. you can retrieve the

6 6
6 6
3 3
4 4
6 1
5 
3 3
5 3
3 3
1 1
1 2
3 4
3 3
3 3
1 1
6 6
3 3
5 5
5 2
3 3
6 1
2 3
2 2
3 2
1 2
60.0
CPU times: user 9min 53s, sys: 1min 14s, total: 11min 7s
Wall time: 6min 59s


In [17]:
count

14